In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Machine_learning").getOrCreate()

23/05/19 04:39:26 WARN Utils: Your hostname, legrand-Aspire-A315-54K resolves to a loopback address: 127.0.1.1; using 192.168.67.128 instead (on interface wlp2s0)
23/05/19 04:39:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/19 04:39:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Chargement du fichier GSE148375_processed_data.txt dans un DataFrame
df_gse = spark.read.csv("GSE148375_processed_data.txt", sep="\t", header=True, inferSchema=True)

In [3]:
from pyspark.sql.functions import col

# Liste des caractères à supprimer
caracteres_a_supprimer = ["DD", "DI", "ID", "II"]

# Filtrage des lignes contenant les caractères à supprimer
for col_name in df_gse.columns:
    df_gse_clean = df_gse.filter(~col(col_name).isin(caracteres_a_supprimer))


In [5]:
#Fusionner toutes les partitions en une seule
df_gse_clean = df_gse_clean.coalesce(1)

# Enregistrer les données dans un seul fichier CSV
df_gse_clean.write.format("csv").option("header", True).option("delimiter", "\t").save("GSE_clean")

In [5]:
import os

def merge_csv_files(file1_path, file2_path, output_file_path):
    # Lecture du contenu du premier fichier
    with open(file1_path, 'r') as file1:
        file1_content = file1.read()
    
    # Lecture du contenu du deuxième fichier
    with open(file2_path, 'r') as file2:
        file2_content = file2.read()
    
    # Fusion des contenus des deux fichiers
    merged_content = file1_content + file2_content
    
    # Écriture du contenu fusionné dans le fichier de sortie
    with open(output_file_path, 'w') as output_file:
        output_file.write(merged_content)

# Chemin des fichiers d'entrée
file1_path = 'GSE_clean/echantillon.csv'
file2_path = 'GSE_clean/echantillon_10.csv'

# Chemin du fichier de sortie
output_file_path = 'GSE_clean/echantillon_finale.csv'

# Fusion des fichiers CSV
merge_csv_files(file1_path, file2_path, output_file_path)


In [6]:
with open('GSE_clean/echantillon_finale.csv', 'r') as input_file:
    # Lecture des lignes du fichier
    lines = input_file.readlines()
    
    # Création d'une liste de listes contenant les valeurs
    values = [line.strip().split('\t') for line in lines]
    
    # Vérification si toutes les lignes ont le même nombre de colonnes
    num_cols = len(values[0])
    if not all(len(row) == num_cols for row in values):
        raise ValueError("Le nombre de colonnes n'est pas cohérent dans le fichier CSV.")
    
    # Initialisation d'une liste de listes pour stocker les colonnes transposées
    transposed = [[] for _ in range(num_cols)]
    
    # Itération sur les lignes pour remplir les colonnes transposées
    for row in values:
        for i in range(num_cols):
            transposed[i].append(row[i])
    
    # Écriture des colonnes transposées dans un nouveau fichier CSV
    with open('GSE_clean/gse_clean_transposed.csv', 'w') as output_file:
        for row in transposed:
            output_file.write('\t'.join(row) + '\n')


In [13]:
df_echantillon = spark.read.csv("GSE_clean/gse_clean_transposed.csv", sep="\t", header=True, inferSchema=True)
df_pheno = spark.read.csv("Phenotype/part-00000-f40e2b0b-25ae-4e3a-84d6-9f2b966e342d-c000.csv", sep="\t", header=True, inferSchema=True)

In [15]:
df_pheno=df_pheno.select("ID", "Smoking_status")

In [16]:
# Jointure des deux DataFrames sur la colonne "ID"
df_join = df_echantillon.join(df_pheno, on="ID")

In [17]:
df_join.show(5)

+-------+-------------------------+--------------------------+--------------------------+--------------------------+---------------------------+---------------------------+---------------------------+---------------------------+--------------------------+--------------------------+--------------+
|     ID|exm62331-0_B_F_1921533749|exm971599-0_T_R_1918203374|exm952264-0_B_F_1918318121|exm101625-0_T_R_1919127660|exm1398889-0_T_F_2060152601|exm1531546-0_T_R_1922618828|exm1271240-0_T_R_1921669182|exm1549624-0_B_F_1922672245|exm234437-0_T_F_1918976439|exm683933-0_T_R_2060422306|Smoking_status|
+-------+-------------------------+--------------------------+--------------------------+--------------------------+---------------------------+---------------------------+---------------------------+---------------------------+--------------------------+--------------------------+--------------+
|1900017|                       GG|                        GG|                        GG|                 

In [19]:
#Fusionner toutes les partitions en une seule
df_join = df_join.coalesce(1)

# Enregistrer les données dans un seul fichier CSV
df_join.write.format("csv").option("header", True).option("delimiter", "\t").save("GSE_clean/dataset")

In [36]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [23]:
# Charger les données
data = pd.read_csv('GSE_clean/dataset/part-00000-24579e76-0819-4ab8-9014-067658729222-c000.csv', sep='\t')

In [24]:
data.head()

,ID,exm62331-0_B_F_1921533749,exm971599-0_T_R_1918203374,exm952264-0_B_F_1918318121,exm101625-0_T_R_1919127660,exm1398889-0_T_F_2060152601,exm1531546-0_T_R_1922618828,exm1271240-0_T_R_1921669182,exm1549624-0_B_F_1922672245,exm234437-0_T_F_1918976439,exm683933-0_T_R_2060422306,Smoking_status
0,1900017,GG,GG,GG,GG,GG,GG,GG,GG,AA,AA,Non-smoker
1,1900029,GG,GG,GG,GG,GG,GG,GG,GG,AA,AA,Smoker
2,1900030,GG,GG,GG,GG,GG,GG,GG,GG,AA,AA,Smoker
3,1900031,GG,GG,GG,GG,GG,GG,GG,GG,AA,AA,Non-smoker
4,1900040,GG,GG,GG,GG,GG,GG,GG,GG,AA,AA,Smoker


In [30]:
# Convertir les génotypes SNP en variables indicatrices (one-hot encoding)
snp_encoded_1 = pd.get_dummies(data['exm62331-0_B_F_1921533749'])
snp_encoded_2 = pd.get_dummies(data['exm971599-0_T_R_1918203374'])
snp_encoded_3 = pd.get_dummies(data['exm952264-0_B_F_1918318121'])
snp_encoded_4 = pd.get_dummies(data['exm101625-0_T_R_1919127660'])
snp_encoded_5 = pd.get_dummies(data['exm1398889-0_T_F_2060152601'])
snp_encoded_6 = pd.get_dummies(data['exm1531546-0_T_R_1922618828'])
snp_encoded_7 = pd.get_dummies(data['exm1271240-0_T_R_1921669182'])
snp_encoded_8 = pd.get_dummies(data['exm1549624-0_B_F_1922672245'])
snp_encoded_9 = pd.get_dummies(data['exm234437-0_T_F_1918976439'])
snp_encoded_10 = pd.get_dummies(data['exm683933-0_T_R_2060422306'])

# Convertir la variable cible en variables indicatrices (one-hot encoding)
label_encoded = pd.get_dummies(data['Smoking_status'])

In [31]:
encoded_df = pd.concat([snp_encoded_1, snp_encoded_2, snp_encoded_3, snp_encoded_4, snp_encoded_5, snp_encoded_6, snp_encoded_7, snp_encoded_8, snp_encoded_9, snp_encoded_10, label_encoded], axis=1)

In [32]:
# Diviser les données en ensemble d'entraînement et ensemble de test
train_data, test_data, train_labels, test_labels = train_test_split(encoded_df.iloc[:, :-1], encoded_df.iloc[:, -1], test_size=0.2, random_state=42)

In [33]:
# Créer et entraîner le modèle SVM
model = svm.SVC(kernel='linear')
model.fit(train_data, train_labels)

SVC(kernel='linear')

In [34]:
# Prédire les labels pour l'ensemble de test
predictions = model.predict(test_data)

In [37]:
# Calculer l'accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


# Avec 100 SNPs

In [39]:
import os

def merge_csv_files(file1_path, file2_path, output_file_path):
    # Lecture du contenu du premier fichier
    with open(file1_path, 'r') as file1:
        file1_content = file1.read()
    
    # Lecture du contenu du deuxième fichier
    with open(file2_path, 'r') as file2:
        file2_content = file2.read()
    
    # Fusion des contenus des deux fichiers
    merged_content = file1_content + file2_content
    
    # Écriture du contenu fusionné dans le fichier de sortie
    with open(output_file_path, 'w') as output_file:
        output_file.write(merged_content)

# Chemin des fichiers d'entrée
file1_path = 'GSE_clean/echantillon.csv'
file2_path = 'GSE_clean/ech_100.csv'

# Chemin du fichier de sortie
output_file_path = 'GSE_clean/echantillon_finale.csv'

# Fusion des fichiers CSV
merge_csv_files(file1_path, file2_path, output_file_path)

In [40]:
with open('GSE_clean/echantillon_finale.csv', 'r') as input_file:
    # Lecture des lignes du fichier
    lines = input_file.readlines()
    
    # Création d'une liste de listes contenant les valeurs
    values = [line.strip().split('\t') for line in lines]
    
    # Vérification si toutes les lignes ont le même nombre de colonnes
    num_cols = len(values[0])
    if not all(len(row) == num_cols for row in values):
        raise ValueError("Le nombre de colonnes n'est pas cohérent dans le fichier CSV.")
    
    # Initialisation d'une liste de listes pour stocker les colonnes transposées
    transposed = [[] for _ in range(num_cols)]
    
    # Itération sur les lignes pour remplir les colonnes transposées
    for row in values:
        for i in range(num_cols):
            transposed[i].append(row[i])
    
    # Écriture des colonnes transposées dans un nouveau fichier CSV
    with open('GSE_clean/gse_clean_transposed_100.csv', 'w') as output_file:
        for row in transposed:
            output_file.write('\t'.join(row) + '\n')


In [41]:
df_echantillon_100=spark.read.csv("GSE_clean/gse_clean_transposed_100.csv", sep="\t", header=True, inferSchema=True)

In [42]:
# Jointure des deux DataFrames sur la colonne "ID"
df_join_100 = df_echantillon_100.join(df_pheno, on="ID")

In [43]:
#Fusionner toutes les partitions en une seule
df_join_100 = df_join_100.coalesce(1)

# Enregistrer les données dans un seul fichier CSV
df_join_100.write.format("csv").option("header", True).option("delimiter", "\t").save("GSE_clean/dataset_100")

In [44]:
# Charger les données
data = pd.read_csv('GSE_clean/dataset_100/part-00000-f5a9ff70-ab1c-4ab6-a97b-8d529a2b3ea3-c000.csv', sep='\t')

In [45]:
data=data.drop('ID', axis=1)

In [46]:
data=pd.get_dummies(data)

In [47]:
encoded_df_100=pd.concat([data],axis=1)

In [49]:
# Diviser les données en ensemble d'entraînement et ensemble de test
train_data, test_data, train_labels, test_labels = train_test_split(encoded_df_100.iloc[:, :-1], encoded_df.iloc[:, -1], test_size=0.2, random_state=42)

In [50]:
# Créer et entraîner le modèle SVM
model = svm.SVC(kernel='linear')
model.fit(train_data, train_labels)

SVC(kernel='linear')

In [51]:
# Prédire les labels pour l'ensemble de test
predictions = model.predict(test_data)

In [52]:
# Calculer l'accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0
